# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
# Import necessary packages
import sqlite3
conn = sqlite3.connect('school.sqlite')

In [2]:
cursor = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [3]:
# Create contactInfo table
cursor.execute('''
CREATE TABLE IF NOT EXISTS contactInfo (
    contact_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    role TEXT NOT NULL CHECK (role IN ('student', 'staff')),
    telephone_number TEXT,
    street TEXT,
    city TEXT,
    state TEXT,
    zipcode TEXT
)
''')

# Commit the changes and close the connection
conn.commit()


## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [4]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [5]:
# Iterate over the contact list and populate the contactInfo table here
# Sample contact data
contact_data = [
    {
        "first_name": "John",
        "last_name": "Doe",
        "role": "student",
        "telephone_number": "555-1234",
        "street": "123 Elm St",
        "city": "Springfield",
        "state": "IL",
        "zipcode": "62701"
    },
    {
        "first_name": "Jane",
        "last_name": "Smith",
        "role": "staff",
        "telephone_number": "555-5678",
        "street": "456 Oak St",
        "city": "Springfield",
        "state": "IL",
        "zipcode": "62702"
    }
    # Add more entries as needed
]

# Reconnect to the database
conn = sqlite3.connect('school.sqlite')
cursor = conn.cursor()

# Insert data into contactInfo table
for contact in contact_data:
    cursor.execute('''
    INSERT INTO contactInfo (first_name, last_name, role, telephone_number, street, city, state, zipcode)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        contact["first_name"],
        contact["last_name"],
        contact["role"],
        contact["telephone_number"],
        contact["street"],
        contact["city"],
        contact["state"],
        contact["zipcode"]
    ))

# Commit the changes and close the connection
conn.commit()

**Query the Table to Ensure it is populated**

In [6]:
# Your code here 
# Query the contactInfo table
cursor.execute('SELECT * FROM contactInfo')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)

(1, 'John', 'Doe', 'student', '555-1234', '123 Elm St', 'Springfield', 'IL', '62701')
(2, 'Jane', 'Smith', 'staff', '555-5678', '456 Oak St', 'Springfield', 'IL', '62702')
(3, 'John', 'Doe', 'student', '555-1234', '123 Elm St', 'Springfield', 'IL', '62701')
(4, 'Jane', 'Smith', 'staff', '555-5678', '456 Oak St', 'Springfield', 'IL', '62702')


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [7]:
# Your code here
# Commit the changes to the database
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [8]:
# Create the grades table
# Create the grades table
cursor.execute('''
CREATE TABLE IF NOT EXISTS grades (
    userId INTEGER NOT NULL,
    courseId INTEGER NOT NULL,
    grade TEXT,
    PRIMARY KEY (userId, courseId)
)
''')
# Commit the changes and close the connection
conn.commit()

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [9]:
# Find the duplicate entry
# Identify duplicate entries based on all columns except the primary key
cursor.execute('''
WITH duplicates AS (
    SELECT 
        first_name, 
        last_name, 
        role, 
        telephone_number, 
        street, 
        city, 
        state, 
        zipcode,
        COUNT(*) as count
    FROM contactInfo
    GROUP BY 
        first_name, 
        last_name, 
        role, 
        telephone_number, 
        street, 
        city, 
        state, 
        zipcode
    HAVING count > 1
)
SELECT * FROM duplicates
''')

# Fetch and print the duplicate entries
duplicates = cursor.fetchall()
print("Duplicate entries:")
for row in duplicates:
    print(row)

# Remove duplicate entries, keeping one instance of each
cursor.execute('''
DELETE FROM contactInfo
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM contactInfo
    GROUP BY 
        first_name, 
        last_name, 
        role, 
        telephone_number, 
        street, 
        city, 
        state, 
        zipcode
)
''')



Duplicate entries:
('Jane', 'Smith', 'staff', '555-5678', '456 Oak St', 'Springfield', 'IL', '62702', 2)
('John', 'Doe', 'student', '555-1234', '123 Elm St', 'Springfield', 'IL', '62701', 2)


In [15]:
# Delete the duplicate entry
# Identify and remove duplicate entries, keeping one instance of each
cursor.execute('''
DELETE FROM contactInfo
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM contactInfo
    GROUP BY 
        first_name, 
        last_name, 
        role, 
        telephone_number, 
        street, 
        city, 
        state, 
        zipcode
)
''')



In [16]:
# Check that the duplicate entry was removed
# Query the contactInfo table to check for duplicates
cursor.execute('''
SELECT first_name, last_name, role, telephone_number, street, city, state, zipcode, COUNT(*)
FROM contactInfo
GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode
HAVING COUNT(*) > 1
''')

# Fetch and print the duplicate entries
duplicates = cursor.fetchall()
if duplicates:
    print("Duplicate entries still exist:")
    for row in duplicates:
        print(row)
else:
    print("No duplicate entries found.")

# Query the contactInfo table to display all entries
cursor.execute('SELECT * FROM contactInfo')
rows = cursor.fetchall()

# Print the results
print("\nCurrent entries in contactInfo table:")
for row in rows:
    print(row)

No duplicate entries found.

Current entries in contactInfo table:
(1, 'John', 'Doe', 'student', '555-1234', '123 Elm St', 'Springfield', 'IL', '62701')
(2, 'Jane', 'Smith', 'staff', '555-5678', '456 Oak St', 'Springfield', 'IL', '62702')


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [17]:
# Update Ed's address
# Update Ed Lyman's address
cursor.execute('''
UPDATE contactInfo
SET street = '2910 Simpson Avenue', city = 'York', state = 'PA', zipcode = '17403'
WHERE first_name = 'Ed' AND last_name = 'Lyman'
''')

In [18]:
# Query the database to ensure the change was made
# Query the contactInfo table to check Ed Lyman's address
cursor.execute('''
SELECT first_name, last_name, street, city, state, zipcode
FROM contactInfo
WHERE first_name = 'Ed' AND last_name = 'Lyman'
''')

# Fetch and print the result
ed_lyman_info = cursor.fetchone()
if ed_lyman_info:
    print("Ed Lyman's updated address:")
    print(ed_lyman_info)
else:
    print("Ed Lyman's record not found.")


Ed Lyman's record not found.


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [19]:
# Your code here
# Commit the changes to the database
conn.commit()

# Close the connection
conn.close()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 